In [ ]:
pip install beautifulsoup4

In [15]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

Escrevendo SCRAPER (Prever erros)

In [16]:
def getTitle(url):
    try:
        html= urlopen(url)
    except HTTPError as e:
        return None
    try:
        bs4= BeautifulSoup(html, 'html.parser')
        title= bs4.html.h1
    except AttributeError as e:
        return None
    return title

title= getTitle('https://github.com/search?o=desc&q=stars%3A%3E100000+language%3APython&s=stars&type=Repositories&l=Python')
if title == None:
    print('Titulo não foi encontrado')
else:
    print(title)



NameError: name 'HTTPError' is not defined

In [34]:
# Configurar o driver do Selenium (certifique-se de ter o driver correspondente instalado)
driver = webdriver.Chrome()

url = 'https://github.com/search?o=desc&q=stars%3A%3E100000+language%3APython&s=stars&type=Repositories&l=Python'
driver.get(url)

# Aguarde alguns segundos para garantir que a página carregue completamente (pode ser ajustado conforme necessário)
time.sleep(5)

# Obtenha o HTML da página
html = driver.page_source
bs4 = BeautifulSoup(html, 'html.parser')

# Encontrar a tag div com a classe específica que contém os resultados
results_divs = bs4.find_all('div', {'class': 'Box-sc-g0xbh4-0 cFPoqW'})

# Criar listas para armazenar os resultados
titles = []
languages = []
stars = []

# Iterar sobre os resultados e extrair as informações
for result_div in results_divs:
    title = result_div.find('div', {'class': 'Box-sc-g0xbh4-0 bBwPjs search-title'}).text.strip()
    language = result_div.find('span', {'class': 'Text-sc-17v1xeu-0 gPDEWA'})
    stars_element = result_div.find('svg', {'class': 'Link__StyledLink-sc-14289xe-0 bzaAZD'})

    # Certificar-se de que todos os elementos foram encontrados antes de tentar acessar o texto
    if language:
        language = language.text.strip()
    else:
        language = 'Not specified'

    if stars_element:
        stars_text = stars_element.text.strip()
    else:
        stars_text = '0'

    titles.append(title)
    languages.append(language)
    stars.append(stars_text)

# Criar um DataFrame do pandas a partir dos resultados
df = pd.DataFrame({'Title': titles, 'Language': languages, 'Stars': stars})

# Imprimir o DataFrame
print(df)

# Fechar o driver do Selenium
driver.quit()

                                  Title Language Stars
0               public-apis/public-apis   Python     0
1      donnemartin/system-design-primer   Python     0
2                  vinta/awesome-python   Python     0
3                  TheAlgorithms/Python   Python     0
4             jackfrued/Python-100-Days   Python     0
5                   ytdl-org/youtube-dl   Python     0
6  AUTOMATIC1111/stable-diffusion-webui   Python     0
7              huggingface/transformers   Python     0


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

In [17]:
url = 'https://github.com/trending'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar os elementos de contêiner para os resultados
    project_containers = soup.find_all('article', {'class': 'Box-row'})

    # Criar lista para armazenar os dados
    data = []

    # Iterar sobre os resultados e extrair as informações
    for container in project_containers:
        # Procurar pelo título do projeto
        title_element = container.find('h2', {'class': 'h3'})
        title = title_element.text.strip() 
        # Remover espaços e quebras de linha extras no título
        title = ' '.join(title.split())

        # Procurar pela linguagem do projeto
        language_element = container.find('span', {'itemprop': 'programmingLanguage'})
        language = language_element.text.strip() if language_element else 'None'

        # Procurar pelo número de estrelas do projeto
        stars_element = container.find('a', {'class': 'Link Link--muted d-inline-block mr-3'})
        stars = stars_element.text.strip() if stars_element else 'None'

        # Procurar pelo número de estrelas hoje
        stars_element_today = container.find('span',{'class':'d-inline-block float-sm-right'})
        stars_today=stars_element_today.text.strip() if stars_element_today else 'None'
        
        # Procurar forks do projeto
        forks_element=container.find('a',{'class':'Link Link--muted d-inline-block mr-3'})
        forks= forks_element.text.strip() if forks_element else 'None'
        #meta_elements = container.find_all('a', {'class': 'Link--muted'})
        #forks = meta_elements[1].text.strip() if len(meta_elements) > 1 else '0' #meta_elements[2].text.strip() if len(meta_elements) > 2 else '0'

        # Adicionar os dados à lista
        data.append([title, language, stars, stars_today, forks])

    # Escrever os dados no arquivo CSV
    with open('github.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=';')
        csv_writer.writerow(['ranking', 'project', 'language', 'stars', 'stars_today', 'forks'])
        
        for index, row in enumerate(data, start=1):
            csv_writer.writerow([index] + row)

    print("Dados extraídos com sucesso e gravados em 'github.csv'.")
else:
    print(f"Falha na requisição com status code: {response.status_code}")


Dados extraídos com sucesso e gravados em 'github.csv'.


In [12]:
# Ler o arquivo CSV substituindo vírgulas por pontos e ponto e vírgula por vírgula
df = pd.read_csv('github.csv', delimiter=';', decimal=',')

# Dividir a coluna "projeto" em duas colunas: "usuário" e "projeto"
df[['usuário', 'projeto']] = df['project'].str.split('/', n=1, expand=True)

# Substituir o ponto e vírgula por vírgula e a vírgula por ponto nas colunas numéricas
df[['stars', 'stars_today']] = df[['stars', 'stars_today']].replace({',': '.', ';': ','}, regex=True)

# Reorganizar as colunas
df = df[['ranking', 'usuário', 'projeto', 'language', 'stars', 'stars_today', 'forks']]

# Salvar o DataFrame de volta como um novo arquivo CSV
df.to_csv('github.csv', index=False, sep=',', decimal='.')
print("Arquivo editado salvo como 'github.csv'.")


Arquivo editado salvo como 'github.csv'.


In [13]:
df= pd.read_csv('github.csv')
display(df)

,ranking,usuário,projeto,language,stars,stars_today,forks
0,1,Ryujinx,Ryujinx,C#,28.815,588 stars today,3.163
1,2,abi,screenshot-to-code,TypeScript,42.333,976 stars today,4.964
2,3,naver,dust3r,Python,2.781,365 stars today,303.000
3,4,vikhyat,moondream,Python,2.432,90 stars today,200.000
4,5,Loyalsoldier,clash-rules,x,15.051,99 stars today,1.267
5,6,dolphin-emu,dolphin,C++,11.731,74 stars today,2.501
6,7,huchenlei,ComfyUI-layerdiffuse,Python,517.000,200 stars today,59.000
7,8,atherosai,ui,HTML,2.552,193 stars today,494.000
8,9,yuzu-mirror,yuzu,C++,1.318,577 stars today,1.003
9,10,PabloMK7,citra,C++,577.000,151 stars today,307.000
